In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

def get_mean_std(df):
    mean_l = [0,0,0]
    std_l = [0,0,0]
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1'])
        band_2 = np.array(row['band_2'])
        band_3 = (band_1 + band_2)/2
        mean_l[0] += np.mean(band_1)
        mean_l[1] += np.mean(band_2)
        mean_l[2] += np.mean(band_3)
        std_l[0] += np.std(band_1)
        std_l[1] += np.std(band_2)
        std_l[2] += np.std(band_3)
    mean_l = np.array(mean_l)/len(df)
    std_l = np.array(std_l)/len(df)
    return mean_l,std_l

mean_l,std_l = get_mean_std(train_df)
print(mean_l,std_l)

[-20.65583096 -26.3207018  -23.48826638] [ 3.10693167  2.64817097  2.36388569]


In [ ]:
def get_image(df,mean_l,std_l):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        
        # preprocess
        band_1 = (band_1 - mean_l[0])/std_l[0]
        band_2 = (band_2 - mean_l[1])/std_l[1]
        band_3 = (band_3 - mean_l[2])/std_l[2]
        

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        images.append(img)
    return np.array(images)


train_x = get_image(train_df,mean_l,std_l)
test_x = get_image(test_df,mean_l,std_l)

print(train_x.shape,test_x.shape)
y = train_df.is_iceberg.values

In [5]:
import pickle
with open('new_preprocess_x_y.pkl','wb') as fout:
    pickle.dump([train_x,y,test_x],fout)
    print('done')

done
